# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [2]:
# Part 1
from sklearn.model_selection import train_test_split
from Utils import train_test_validation
import numpy as np

# Define the percentages
train_percent = 0.60
test_percent = 0.15
validation_percent = 0.25

#num_samples = len(data['X'])
#train_samples = int(train_percent * num_samples)
#test_samples = int(test_percent * num_samples)
#validation_samples = num_samples - train_samples - test_samples

#X_temp, X_test, y_temp, y_test = train_test_split(X, Y, test_size=test_samples, shuffle=False)
#new_validation_percent = validation_samples/len(X_temp)
#X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=new_validation_percent, shuffle=False)
#X_train, X_test, X_valid, Y_train, Y_test, Y_valid = X_train, X_test, X_val, y_train, y_test, y_val

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = train_test_validation(X,Y,test_size=test_percent,validation_size=validation_percent,random_state=None,shuffle=False)

print("shape X_train: ", np.shape(X_train))
print("shape X_test: ", np.shape(X_test))
print("shape X_valid: ", np.shape(X_valid))
print("shape Y_train: ", np.shape(Y_train))
print("shape Y_test: ", np.shape(Y_test))
print("shape Y_valid: ", np.shape(Y_valid))


# bennys ----
# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)



shape X_train:  (6709, 6)
shape X_test:  (1677, 6)
shape X_valid:  (2797, 6)
shape Y_train:  (6709,)
shape Y_test:  (1677,)
shape Y_valid:  (2797,)


In [3]:

# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression

import scipy.io as so
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Create a pipeline with StandardScaler and Logistic Regression
model = Pipeline([
    ('scaler', StandardScaler()),
    #('classifier', LogisticRegression())
    ('model', LogisticRegression())
])

# Train the model on the training data
model.fit(X_train, Y_train)

# Make predictions on the validation set
predictions = model.predict(X_test)

# Evaluate the accuracy on the validation set
#accuracy = accuracy_score(y_test, predictions)
#print("Validation Accuracy:", accuracy)

#model = model

In [4]:

# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

import re

# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)

alphahoff= 0.05

report = classification_report_interval(Y_test, predictions,alpha=alphahoff)
#print(report)

precision0 = (0.94,1.00)
recall0 = (0.96,1.00)
precision1 = (0.33,1.00)
recall1 = (0.09,0.62)

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

In [5]:

# Part 4

def cost(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost (cost per person)
    
    cost = 0
    FP = 0
    FN = 0
    TP = 0
    TN = 0
    
    #X_train, X_test, X_valid, Y_train, Y_test, Y_valid
    #if someone is calcified but classified as not 
    for i in range(len(Y)):
        if (Y[i] ==1 and predictions[i] == 0): #FP - calcified but the results says that you're not
            FP += 1
        if(Y[i]==0 and predictions[i] ==1): #FN - is not calcified but the results says that you are
            FN +=1
        if(Y[i] == 1 and predictions[i]==1): #TP, du är calcified och resultaten visar det
            TP +=1
        if(Y[i] ==0 and predictions[i] ==0 ): #du är inte calcified och resultaten visar det
            TN +=1
    
    cost = 30*FP + 5*FN
    average_cost_per_person = cost/len(Y)
    
    return average_cost_per_person

In [6]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set

threshold = 1.1
naive_cost = cost(model, threshold, X, Y)
#print(naive_cost) #gives all points, is within 0.01
#print(0.6976744186046512) #correct answer

In [7]:

# Part 5
import numpy as np

spaced_thresholds = np.linspace(0, 1, num=100)

optimal_cost = 0
current_cost = cost(model, spaced_thresholds[0], X, Y)
optimal_threshold = spaced_thresholds[0]
#current_cost = float('inf')
#optimal_threshold = 0

cost_array = []

for threshold in spaced_thresholds:
    current_threshold_cost = cost(model, threshold, X_test, Y_test)
    cost_array.append(current_threshold_cost)
    
    if  current_threshold_cost < current_cost:
        current_cost = current_threshold_cost
        optimal_threshold = threshold
        optimal_cost = current_threshold_cost
        

optimal_threshold = optimal_threshold
cost_at_optimal_threshold = optimal_cost

print(optimal_threshold)
print("Optimal cost: ", cost_at_optimal_threshold)

0.11111111111111112
Optimal cost:  0.3518187239117472


In [8]:
# Part 6
#from Utils import compute_confidence_interval_bounded
from Utils import epsilon_bounded
#from sklearn.metrics import mean_absolute_error
import numpy as np

#calculate optimal cost with validation data
cost_at_optimal_threshold_validation = cost(model, optimal_threshold, X_valid, Y_valid)
print("cost_at_optimal_threshold_validation: ", cost_at_optimal_threshold_validation)

n = len(X_valid)
b = 30
a = 0 #not used
alpha = 0.05
epsilon = epsilon_bounded(n,b,alpha)
print("epsilon: ", epsilon)

point_estimate = cost_at_optimal_threshold_validation #estimated cost at optimal thresh
#print("point_estimate: ", point_estimate)
r_edge = np.minimum(point_estimate+epsilon,b)
#print("r_edge: ",r_edge)

# Report the cost interval as a tuple cost_interval = (a,b)
cost_interval = (0.0, 1.0617676271684535)
print("cost_interval: ", cost_interval)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)


cost_at_optimal_threshold_validation:  0.2913836253128352
epsilon:  0.7703840018556183
cost_interval:  (0.0, 1.0617676271684535)


In [9]:
# Part 7

def C_var(model,threshold,X,Y):
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    c_cost_array = []
    
    cost = 0
    
    for i in range(len(Y)):
        FP = 0
        FN = 0
        TP = 0
        TN = 0
        if (Y[i] ==1 and predictions[i] == 0): #FP - calcified but the results says that you're not
            FP += 1
        if(Y[i]==0 and predictions[i] ==1): #FN - is not calcified but the results says that you are
            FN +=1
        if(Y[i] == 1 and predictions[i]==1): #TP, du är calcified och resultaten visar det
            TP +=1
        if(Y[i] ==0 and predictions[i] ==0 ): #du är inte calcified och resultaten visar det
            TN +=1
    
        cost = 30*FP + 5*FN
        c_cost_array.append(cost)
    
    return c_cost_array

c_cost_array = C_var(model, optimal_threshold, X_valid, Y_valid)
  

empirical_variance = np.var(c_cost_array)

variance_of_C = empirical_variance
print("variance_of_C:", variance_of_C)

variance_of_C: 6.198613637958541


In [10]:
# Part 7
from Utils import bennett_epsilon

n_valid = len(X_valid)

print(n_valid)
b=30
sigma = np.sqrt(variance_of_C)
alpha = 0.05
bennett = bennett_epsilon(n,b,sigma,alpha)
#print("bennett: ", bennett)

bennett_interval = (cost_at_optimal_threshold_validation - bennett,
                    cost_at_optimal_threshold_validation + bennett)

print("BENNYS Bennett's Confidence Interval:", bennett_interval)


# A 95% confidence interval of the random variable C using Bennett's inequality
interval_of_C = (0.15091432947852831, 0.431852921147142)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

2797
Numerical error -1.1102230246251565e-16
BENNYS Bennett's Confidence Interval: (0.15091432947852831, 0.431852921147142)
